In [8]:
%config InlineBackend.rc = {"figure.dpi": 72, 'figure.figsize': (6.0, 4.0)}
%matplotlib ipympl

import abtem
import ase
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML

abtem.config.set({"local_diagnostics.progress_bar": True})
abtem.config.set({"device": "cpu"});

# Interactive visualizations

## Probe aberrations

In [12]:
probe = abtem.Probe(
    semiangle_cutoff=30,
    defocus=np.linspace(-70, 70, 30),
    astigmatism=np.linspace(-70, 70, 30),
    astigmatism_angle=np.pi / 4,
    gpts=256,
    extent=10,
    energy=200e3,
)
probes = probe.build().intensity().compute()  # .compute()

[########################################] | 100% Completed | 918.44 ms


In [13]:
visualization = probes.show(cmap="viridis", display=False, cbar=True, interact=True)

## HRTEM

In [4]:
tube1 = ase.build.nanotube(10, 0, length=5)
tube2 = ase.build.nanotube(16, 0, length=5)
tubes = tube1 + tube2
tubes.center(vacuum=4.0, axis=(0, 1))

rotated_tubes = tubes.copy()
rotated_tubes.rotate("y", 90, rotate_cell=True)
rotated_tubes = abtem.standardize_cell(rotated_tubes)

potential = abtem.Potential(
    rotated_tubes,
    sampling=0.05,
    projection="infinite",
    slice_thickness=1,
    exit_planes=1,
)

# abtem.show_atoms(tubes, plane="xy", title="Beam view");

In [ ]:
wave = abtem.PlaneWave(energy=100e3)

exit_wave = wave.multislice(potential).intensity().compute()

In [5]:
visualization = exit_wave[:].show(interact=True, axes_types=["index"])

NameError: name 'exit_wave' is not defined

## STEM

In [14]:
atoms = ase.build.mx2(vacuum=3)
atoms = abtem.orthogonalize_cell(atoms) * (3, 2, 1)

probe = abtem.Probe(
    semiangle_cutoff=np.linspace(20, 30, 3),
    defocus=np.linspace(0, 50, 3),
    Cs=np.linspace(0, 5e5, 3),
    astigmatism_angle=np.pi / 4,
    sampling=0.05,
    energy=80e3,
)

potential = abtem.Potential(
    atoms, projection="infinite", slice_thickness=2
)

scan = abtem.GridScan(
    start=(0, 0),
    end=(1 / 3, 1 / 2),
    fractional=True,
    potential=potential,
)

doses = np.geomspace(1e1, 1e7, 10)

measurements = (
    probe.scan(potential, scan=scan)
    .to_images() # to ensemble
    .gaussian_filter(0.3)
    .interpolate(0.1)
    .tile((3, 2))
    .poisson_noise(dose_per_area=doses)
    .compute()
)

[########################################] | 100% Completed | 27.58 s


In [15]:
visualization = measurements[:, :, :, :, :, 0].show(
    interact=True, axes_types=["index"] * 4 + ["range"], cbar=True
)